In [36]:
import sqlite3
import pandas as pd
conn = sqlite3.connect("..//sqlite_DB//Movie_report.db")
cur = conn.cursor()
select_movie_fact ="""with last_six_months as (
select Month from DateDim where Month <=200010 order by Month desc limit 6),
max_review_count as (
select 
max(a.Rating_monthly_count)/2  max_count,
Review_month from 
  Movie_ratings_monthly a,last_six_months b
where a.Review_month =b.Month
group by Review_month),
movie_six_months as
(select a.MovieID,
a.Rating_monthly_count,
a.Review_month,
a.Rating_monthly_avg,
a.OverallCount,
a.Overall_rating_avg,
row_number() over(PARTITION by a.Review_month order by Rating_monthly_avg desc) rnk
from
Movie_ratings_monthly a,max_review_count c
where a.Review_month=c.Review_month
and a.Rating_monthly_count >= c.max_count)
select
a.MovieID, 
a.rnk ,
b.Title,
a.Rating_monthly_count,
a.Review_month,
a.Rating_monthly_avg,
a.OverallCount,
a.Overall_rating_avg
 from movie_six_months a,MovieDIM b
 where a.MovieID =b.MovieID and 
  a.rnk <= 10
  order by b.Title,a.Review_month desc
"""
df_details = pd.read_sql_query(select_movie_fact, conn)
conn.close()
df_details

,MovieID,rnk,Title,Rating_monthly_count,Review_month,Rating_monthly_avg,OverallCount,Overall_rating_avg
0,2858,9,American Beauty (1999),194,200009,4.44,1556,4.366
1,2858,8,American Beauty (1999),220,200006,4.36,454,4.350
2,2997,10,Being John Malkovich (1999),144,200005,4.38,157,4.380
3,541,5,Blade Runner (1982),120,200006,4.44,286,4.430
4,541,7,Blade Runner (1982),134,200005,4.42,166,4.430
5,912,7,Casablanca (1942),70,200010,4.47,899,4.438
6,912,1,Casablanca (1942),193,200007,4.60,472,4.462
7,912,8,Casablanca (1942),149,200005,4.40,170,4.400
8,858,6,"Godfather, The (1972)",94,200010,4.47,1185,4.520
9,858,10,"Godfather, The (1972)",117,200009,4.42,1091,4.524


In [2]:
conn = sqlite3.connect("..//sqlite_DB//Movie_report.db")
cur = conn.cursor()
last_6_months ="""
select Month from DateDim where Month <=200010 order by Month desc limit 6
"""
last_6_months_df = pd.read_sql_query(last_6_months, conn)
conn.close()
last_6_months_df

,Month
0,200010
1,200009
2,200008
3,200007
4,200006
5,200005


In [9]:
for index,row in last_6_months_df.iterrows():
    print(index,row['Month'])

0 200010
1 200009
2 200008
3 200007
4 200006
5 200005


In [20]:
df_details_latest =df_details[df_details['Review_month'] =='200010'].copy()

In [33]:
for index,row in df_details_latest.iterrows():
    print(row['Title'])
    Movieid =row['MovieID']
    cnt =0
    for index_m,row_m in last_6_months_df.iterrows():
        if index_m > 0:            
            df_details_temp =df_details[(df_details['Review_month'] ==str(row_m['Month'])) & (df_details['MovieID'] ==Movieid)].copy()
            if len(df_details_temp) > 0:
                cnt =cnt +1
            else:
                break
    print(index,row_m['Month'],cnt)

Godfather: Part II, The (1974)
0 200009 0
Shawshank Redemption, The (1994)
1 200005 5
Monty Python and the Holy Grail (1974)
2 200009 0
Princess Bride, The (1987)
3 200009 0
One Flew Over the Cuckoo's Nest (1975)
4 200008 1
Godfather, The (1972)
5 200005 5
Casablanca (1942)
6 200009 0
Raiders of the Lost Ark (1981)
7 200005 4
Sixth Sense, The (1999)
8 200006 3
Usual Suspects, The (1995)
9 200006 3


In [ ]:



for index,row in last_6_months_df.iterrows():
    print(index,row['Month'])
